# Pruebas análisis de datos

En el presente documentos se hace una síntesis del análisis que se ha llevado a cabo sobre los datos de AirBnB y las diferentes pruebas que se han hecho sobre estos para utilizarlos para el entrenamiento del modelo.

In [123]:
## Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re # Para obtener la superficie de la descripción de los apartamentos

## Análisis de los datos iniciales

A continuación, se presentan los diferentes análisis realizados sobre los datos presentes en el _data-set_ de AirBnB.

In [124]:
df = pd.read_csv('../data/listings.csv')

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            25288 non-null  int64  
 1   listing_url                                   25288 non-null  object 
 2   scrape_id                                     25288 non-null  int64  
 3   last_scraped                                  25288 non-null  object 
 4   source                                        25288 non-null  object 
 5   name                                          25288 non-null  object 
 6   description                                   24375 non-null  object 
 7   neighborhood_overview                         11218 non-null  object 
 8   picture_url                                   25287 non-null  object 
 9   host_id                                       25288 non-null 

In [126]:
df.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,2.528800e+04,2.528800e+04,2.528800e+04,25269.000000,25269.000000,25288.000000,25288.000000,25288.000000,19270.000000,22737.000000,...,20090.000000,20090.000000,20091.000000,20088.000000,20087.000000,25288.000000,25288.000000,25288.000000,25288.000000,20091.000000
mean,6.439293e+17,2.025031e+13,2.583530e+08,53.929400,63.880169,40.421572,-3.693851,3.159206,1.286897,1.437173,...,4.678302,4.742479,4.747868,4.747256,4.549966,36.220263,32.311768,3.836958,0.042036,1.778151
std,5.357949e+17,0.000000e+00,2.083118e+08,144.357627,193.134128,0.023441,0.027972,1.905684,0.628857,0.995338,...,0.456137,0.448347,0.474671,0.383183,0.518818,81.239399,80.203422,16.249609,0.589461,1.914170
min,2.185300e+04,2.025031e+13,7.952000e+03,1.000000,1.000000,40.331400,-3.883990,1.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,3.676095e+07,2.025031e+13,5.168513e+07,1.000000,2.000000,40.409288,-3.707420,2.000000,1.000000,1.000000,...,4.590000,4.700000,4.720000,4.670000,4.440000,1.000000,0.000000,0.000000,0.000000,0.360000
50%,8.115619e+17,2.025031e+13,2.228602e+08,4.000000,5.000000,40.420382,-3.700932,2.000000,1.000000,1.000000,...,4.800000,4.860000,4.890000,4.870000,4.670000,3.000000,1.000000,0.000000,0.000000,1.130000
75%,1.142574e+18,2.025031e+13,4.488906e+08,25.000000,33.000000,40.431617,-3.684771,4.000000,1.500000,2.000000,...,4.950000,4.990000,5.000000,4.980000,4.840000,19.000000,12.000000,1.000000,0.000000,2.630000
max,1.369179e+18,2.025031e+13,6.821759e+08,3311.000000,8554.000000,40.577290,-3.545904,16.000000,15.000000,25.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,341.000000,341.000000,168.000000,12.000000,28.570000


### Limpieza inicial de datos

#### Datos inútiles

A primera vista ya se ve que hay una gran cantidad de datos que, aunque puedan afectar al precio, no existen dentro del _data-set_ de Idealista.

Procedemos a hacer una primera limpiaeza borrando dichos valores.

In [127]:
dropping_cols = ["scrape_id", 
                 "last_scraped", 
                 "source", # Datos sobre scrapping innecesarios
                 "neighborhood_overview", # Descripción de la zona por el propietario con muchos valores nulos
                 "picture_url", # Imagen del apartamento
                 "host_id",
                 "host_url", 
                 "host_name", 
                 "host_since", 
                 "host_location", 
                 "host_about", 
                 "host_thumbnail_url", 
                 "host_response_time",
                 "host_response_rate",
                 "host_acceptance_rate",
                 "host_is_superhost",
                 "host_picture_url", 
                 "host_neighbourhood", 
                 "host_listings_count", 
                 "host_total_listings_count", 
                 "host_verifications", 
                 "host_has_profile_pic", 
                 "host_identity_verified", # Datos del propietario inútiles para el entrenamiento del model
                 "neighbourhood", # Etiquelas con poco valor, repetidas en su mayoría de diferentes formas
                 "neighbourhood_cleansed", # Etiquetas con las que no contamos en data-set de Idealista. Con coordenadas y neighbourhood_group_cleansed tenemos toda la información sobre localización necesaria
                 "bathrooms_text", # En data-set de Idealista solo tenemos la cantidad de baños, no su descripción
                 "beds", # Número de habitaciones de mayor utilidad
                 "minimum_nights",
                 "maximum_nights",
                 "minimum_minimum_nights",
                 "minimum_maximum_nights",
                 "maximum_minimum_nights",
                 "maximum_maximum_nights",
                 "minimum_nights_avg_ntm",
                 "maximum_nights_avg_ntm", # Datos dependientes de los propietarios, no de las viviendas
                 "calendar_updated",
                 "has_availability",
                 "availability_30",
                 "availability_60",
                 "availability_90",
                 "availability_365", 
                 "calendar_last_scraped", # Datos sobre disponibilidad. Sin utilidad para el modelo
                 "number_of_reviews",
                 "number_of_reviews_ltm",
                 "number_of_reviews_l30d",
                 "availability_eoy",
                 "number_of_reviews_ly",
                 "estimated_revenue_l365d",
                 "estimated_occupancy_l365d", # Estimaciones que no tenemos claro de dónde salen ni cómo las han calculado. Por lo quue no lo utilizamos
                 "first_review",
                 "last_review",
                 "review_scores_rating",
                 "review_scores_accuracy",
                 "review_scores_cleanliness",
                 "review_scores_checkin",
                 "review_scores_communication",
                 "review_scores_location",
                 "review_scores_value", # Datos sobre reviews, pueden afectar al precio pero no son últiles para el entrenamiento del modelo
                 "license", # Datos de licencia inútiles
                 "instant_bookable",
                 "calculated_host_listings_count",
                 "calculated_host_listings_count_entire_homes",
                 "calculated_host_listings_count_private_rooms",
                 "calculated_host_listings_count_shared_rooms",
                 "reviews_per_month"] # Información sobre los anuncios en sí y los propietarios que no afectan al precio por noche de los apartamentos

df = df.drop(dropping_cols, axis=1)

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            25288 non-null  int64  
 1   listing_url                   25288 non-null  object 
 2   name                          25288 non-null  object 
 3   description                   24375 non-null  object 
 4   neighbourhood_group_cleansed  25288 non-null  object 
 5   latitude                      25288 non-null  float64
 6   longitude                     25288 non-null  float64
 7   property_type                 25288 non-null  object 
 8   room_type                     25288 non-null  object 
 9   accommodates                  25288 non-null  int64  
 10  bathrooms                     19270 non-null  float64
 11  bedrooms                      22737 non-null  float64
 12  amenities                     25288 non-null  object 
 13  p

#### Anuncios sin precio

El valor _target_ que utilizaremos para el modelo será el precio, por lo que borraremos todos aquellos anuncios que no tengan datos sobre el precio.

In [129]:
df = df.dropna(subset='price')

#### Anuncios repetidos

También hay que borrar todos los anuncios que se encentren duplicados. Para ello, utilizaremos las columnas de _name_ y _description_.

In [130]:
df["name_description"] = df["name"].fillna(" ").str.lower() + " and " + df["description"].fillna(" ").str.lower()
df["name_description"].groupby(df["name_description"]).count()[df["name_description"].groupby(df["name_description"]).count() > 1].sort_values(ascending=False)

name_description
fantastic residence ``funway´´ north zone madrid and modern and bright space in the financial district of madrid, only 20 minutes away from the city center.<br />perfectly communicated with lines 1 (tetuan) and 10 ( cuzco ) of metro and various bus lines.<br />very near to chamartín and nuevos ministerios stations.                                                                                                                                                                                                                                  18
flamenco style apartment | palacio real, la latina and wonderful apartment located just above one of the oldest flamenco tablao in madrid, the corral de la morería, recently renovated opting for quality and comfort, offering an incredible rest. located in one of the most central areas of madrid that offers a multitude of restaurants, leisure and culture a few steps from the nerve center of the city. it has a bedroom with a queen size 

Observamos que, efectivamente, hay varios anuncios que se encuentran repetidos. Borramos todos los repetidos menos uno.

In [131]:
df.drop_duplicates(subset=["name_description"], keep="first", inplace=True)

Se pueden borrar las columnas de _name_ y _description_ dado que ya hemos creado una columna que cuenta con toda esta información, la cual utilizaremos más adelante para obtener más datos.

In [132]:
df = df.drop(['name', 'description'], axis=1)

#### Anuncios que no sean de viviendas completas

Dado que en el _data-set_ de Idealista solo hay datos sobre viviendas completas, no tiene sentido entrenar el modelo con otros tipos de anuncios que no sean viviendas completas. 

Es por ello que borramos todos los anuncios que no cumplan con esta condición.

In [133]:
df = df[df['room_type']== 'Entire home/apt']

In [134]:
property_types_to_remove = ['Camper/RV', 'Room in aparthotel', 'Entire cabin','Hut','Yurt','Entire Bungalow']
df = df[~df['property_type'].isin(property_types_to_remove)]

Hecho esto, podemos borrar las columnas de _room_type_ y de _property_type_ dado que ya no ofrecen ninguna información de valor.

In [135]:
df = df.drop(['room_type', 'property_type'], axis=1)

### Cambio en tipos de datos

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13632 entries, 5 to 25285
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            13632 non-null  int64  
 1   listing_url                   13632 non-null  object 
 2   neighbourhood_group_cleansed  13632 non-null  object 
 3   latitude                      13632 non-null  float64
 4   longitude                     13632 non-null  float64
 5   accommodates                  13632 non-null  int64  
 6   bathrooms                     13631 non-null  float64
 7   bedrooms                      13627 non-null  float64
 8   amenities                     13632 non-null  object 
 9   price                         13632 non-null  object 
 10  name_description              13632 non-null  object 
dtypes: float64(4), int64(2), object(5)
memory usage: 1.2+ MB


#### Baños, habitaciones

In [137]:
df['bedrooms'].unique()

array([ 1.,  3.,  0.,  2.,  4.,  5.,  6.,  8.,  7., nan,  9.])

In [138]:
df['bedrooms'].unique()

array([ 1.,  3.,  0.,  2.,  4.,  5.,  6.,  8.,  7., nan,  9.])

Tanto en baños como en habitaciones observamos que hay tanto apartamentos con 0 unidades de cada uno como valores desconocidos. Esto seguramente sea porque se trate de apartamentos en los que, o bien no se ha especificado el dato, o bien se trata de apartamentos tipo _loft_. En todo caso, es lógico pensar que cualquier vivienta cuenta, por lo menos, con 1 baño y 1 habitación. Por lo que cambiaremos los valores nulos y los desconocidos por 1.

Además, el tipo de dato con el que están guardados dichos datos es de tipo _float_ cuando claramente se trata de datos tipo _int_. Por lo que también cambiaremos esto.

In [139]:
df['bathrooms'] = df['bathrooms'].replace([np.inf,-np.inf],np.nan).fillna(0).astype(int)
df['bedrooms'] = df['bedrooms'].replace([np.inf,-np.inf],np.nan).fillna(0).astype(int)

df['bathrooms'] = df['bathrooms'].replace(0,1)
df['bedrooms'] = df['bedrooms'].replace(0,1)

#### Precio

In [140]:
df['price'].unique()

array(['$66.00', '$89.00', '$173.00', '$72.00', '$189.00', '$94.00',
       '$90.00', '$87.00', '$82.00', '$170.00', '$351.00', '$229.00',
       '$177.00', '$86.00', '$57.00', '$88.00', '$118.00', '$114.00',
       '$100.00', '$160.00', '$71.00', '$110.00', '$138.00', '$91.00',
       '$131.00', '$106.00', '$64.00', '$129.00', '$63.00', '$243.00',
       '$62.00', '$277.00', '$144.00', '$84.00', '$120.00', '$95.00',
       '$97.00', '$70.00', '$80.00', '$101.00', '$141.00', '$259.00',
       '$75.00', '$102.00', '$116.00', '$98.00', '$67.00', '$166.00',
       '$127.00', '$152.00', '$96.00', '$190.00', '$300.00', '$109.00',
       '$76.00', '$176.00', '$78.00', '$148.00', '$204.00', '$132.00',
       '$65.00', '$85.00', '$154.00', '$222.00', '$225.00', '$169.00',
       '$73.00', '$126.00', '$214.00', '$145.00', '$136.00', '$143.00',
       '$108.00', '$209.00', '$159.00', '$119.00', '$83.00', '$122.00',
       '$149.00', '$202.00', '$112.00', '$39.00', '$459.00', '$93.00',
       '$4

Observamos que los datos de precios por noche están guardados como tipo _object_ al ser _strings_ con el símbolo del dolar.

Vamos a pasarlos a valores numéricos tipo _float_.

In [141]:
df['price'] = df['price'].replace({'\\$': '', ',': ''}, regex=True).astype(float)

### Extracción de datos de interés 

Hay algunas variables que nos ofrecen información valiosa pero que no se puede aprovechar tal y como aparece en el _data-set_.

Es por eso que, a continuación, extraeremos todos los datos que nos puedan ser se utilidad para el entrenamiento del modelo de las columnas:
- _name_description_
- _amenities_


#### _name_description_

De esta variable podemos obtener los siguientes datos:
- Terraza
- Balcón
- Jardín
- Superficie del apartamento en m2

In [142]:
# Terraza
df["terraza"] = df["name_description"].apply(lambda x: 1 if "terrace" in x else 0)
# Balcón
df["balcon"] = df["name_description"].apply(lambda x: 1 if "balcon" in x else 0)
# Jardín
df["jardin"] = df["name_description"].apply(lambda x: 1 if "garden" in x else 0)

In [143]:
# Superficie
def get_meters(data):
    """
    Devuelve un set con todos los metros únicos presentes en la columna 'name_description' del DataFrame.
    """
    matches_1 = re.findall(r'(\d+)\s?m²', data)
    matches_2 = re.findall(r'(\d+)\s?m2', data)
    if matches_1:
        return int(matches_1[0])
    if matches_2:
        return int(matches_2[0])

    return None

df["meters"] = df["name_description"].apply(get_meters)

Borramos la columna de _name_description_ al haber extraído todas las variables que nos eran de utilidad.

In [144]:
df = df.drop('name_description', axis=1)

#### _amenities_

De esta variable, podemos obtener los siguientes datos:
- Calefacción
- Piscina
- Aparcamiento
- Aire acondicionado
- Ascensor
- Instalaciones para personas con movilidad reducida

In [145]:
# Calefacción
df['calefaccion'] = df['amenities'].apply(lambda x: 1 if 'heating' in x.lower() else 0)
# Piscina
df['pool'] = df['amenities'].apply(lambda x: 1 if ('pool' in str.lower(x)) and ('whirlpool' not in str.lower(x)) and ('whirpool' not in str.lower(x)) else 0)
# Aparcamiento
df['garaje'] = df['amenities'].apply(lambda x: 1 if 'free parking' in x.lower() else 0)
# Aire acondicionado
df['aire_acondicionado'] = df['amenities'].apply(lambda x: 1 if 'air conditi' in str.lower(x) else 0)
# Ascensor
df['ascensor'] = df['amenities'].apply(lambda x: 1 if 'elevator' in str.lower(x) else 0)
# Movilidad reducida
df['movilidad_reducida'] = df['amenities'].apply(lambda x: 1 if 'single level home' in str.lower(x) else 0)

En el caso de la movilidad reducida, no tiene sentido que se tome con movilidad reducida aquellos apartamentos que no cuentan con ascensor.

In [146]:
len(df)

13632

In [ ]:
df.loc[(df['movilidad_reducida'] == 1) & (df['ascensor'] == 0)]

In [147]:
df.loc[(df['movilidad_reducida'] == 1) & (df['ascensor'] == 0), "movilidad_reducida"] = 0

In [148]:
len(df)

13632

In [149]:
df.loc[(df['movilidad_reducida'] == 1) & (df['ascensor'] == 0)]

,id,listing_url,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,amenities,price,terraza,balcon,jardin,meters,calefaccion,pool,garaje,aire_acondicionado,ascensor,movilidad_reducida
